# Week3. Homework.

## Author George Chizhmak (02.10.2023).

Dataset
In this homework, we will use the Car price dataset. Download it from here [https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv].

We'll keep working with the MSRP variable, and we'll transform it to a classification task.

## Features
For the rest of the homework, you'll need to use only these columns:

Make,
Model,
Year,
Engine HP,
Engine Cylinders,
Transmission Type,
Vehicle Style,
highway MPG,
city mpg,
MSRP

In [1]:
import pandas as pd
import numpy as np

In [104]:
data = pd.read_csv("./data.csv")

In [105]:
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [106]:
data = data[['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']]

## Data preparation

Select only the features from above and transform their names using the next line:
data.columns = data.columns.str.replace(' ', '_').str.lower()
Fill in the missing values of the selected features with 0.
Rename MSRP variable to price.

In [107]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [108]:
data.fillna(0, inplace=True)

In [109]:
data.rename(columns={"msrp": "price"}, inplace=True)

## Question 1
What is the most frequent observation (mode) for the column transmission_type?

In [110]:
data['transmission_type'].value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

In [111]:
data.corr(numeric_only=True)

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


## Make price binary

Now we need to turn the price variable from numeric into a binary format.
Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

In [112]:
data_with_price = data.copy()

mean_price = data['price'].mean()
data['above_average'] = data['price'].apply(lambda x: 1 if x > mean_price else 0)

del data['price']

## Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value (above_average) is not in your dataframe.

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=42)

In [50]:
df_train, df_val = train_test_split(df_train_full, test_size=0.2, random_state=42)

In [51]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values

In [52]:
del df_train['above_average']
del df_val['above_average']

## Question 3

- Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).
- Which of these variables has the lowest mutual information score?

In [53]:
from sklearn.metrics import mutual_info_score

In [54]:
def mutual_info_above_average(series):
    return round(mutual_info_score(series, y_train), 2)

In [55]:
df_train.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
dtype: object

In [56]:
categorical = [
    'make',
    'model',
    'transmission_type',
    'vehicle_style'
]

In [57]:
mi = df_train[categorical].apply(mutual_info_above_average)
mi.sort_values(ascending=True)

transmission_type    0.02
vehicle_style        0.08
make                 0.24
model                0.46
dtype: float64

## Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

In [58]:
from sklearn.feature_extraction import DictVectorizer

In [59]:
train_dict = df_train.to_dict(orient='records')

In [60]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [61]:
X_train = dv.transform(train_dict)

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [64]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
y_pred = model.predict(X_val)

score =accuracy_score(y_val, y_pred)

In [71]:
round(score, 2)

0.94

In [68]:
# y_pred = model.predict_proba(X_val)[:, 1]
# churn = y_pred > 0.5
# (y_val == churn).mean()

## Question 5

- Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [77]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'city_mpg': 0.084,
 'engine_cylinders': -0.055,
 'engine_hp': 0.036,
 'highway_mpg': 0.0,
 'make=Acura': 1.135,
 'make=Alfa Romeo': 1.289,
 'make=Aston Martin': 0.448,
 'make=Audi': 3.19,
 'make=BMW': 2.477,
 'make=Bentley': 0.136,
 'make=Bugatti': 0.0,
 'make=Buick': -0.429,
 'make=Cadillac': 2.478,
 'make=Chevrolet': -1.581,
 'make=Chrysler': -1.266,
 'make=Dodge': -2.977,
 'make=FIAT': -0.409,
 'make=Ferrari': 0.26,
 'make=Ford': -2.206,
 'make=GMC': -0.934,
 'make=Genesis': 0.367,
 'make=HUMMER': -0.073,
 'make=Honda': -1.164,
 'make=Hyundai': -2.56,
 'make=Infiniti': 0.355,
 'make=Kia': -1.521,
 'make=Lamborghini': 0.007,
 'make=Land Rover': 2.0,
 'make=Lexus': 1.283,
 'make=Lincoln': 1.077,
 'make=Lotus': 3.59,
 'make=Maserati': 0.819,
 'make=Maybach': 0.003,
 'make=Mazda': -1.554,
 'make=McLaren': 0.0,
 'make=Mercedes-Benz': 0.879,
 'make=Mitsubishi': -1.557,
 'make=Nissan': -1.216,
 'make=Oldsmobile': -1.02,
 'make=Plymouth': -0.152,
 'make=Pontiac': -2.523,
 'make=Porsche': 1

In [90]:
original_accuracy = score

# Initialize variables to track the smallest absolute difference and the corresponding feature
smallest_difference = float('inf')
least_useful_feature = None

# Initialize a dictionary to store scores for each feature
feature_scores = {}

# Iterate through each feature and calculate the difference in accuracy
for feature in dv.feature_names_:
    # Find the index of the feature in the feature names
    feature_index = dv.feature_names_.index(feature)

    # Exclude the current feature from both X_train and X_val
    X_train_without_feature = np.delete(X_train, feature_index, axis=1)
    X_val_without_feature = np.delete(X_val, feature_index, axis=1)

    # Train a model without the current feature
    model_without_feature = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_without_feature.fit(X_train_without_feature, y_train)
    y_pred_without_feature = model_without_feature.predict(X_val_without_feature)

    accuracy_without_feature = accuracy_score(y_val, y_pred_without_feature)
    difference = abs(original_accuracy - accuracy_without_feature)

    # Store the accuracy difference in the dictionary
    feature_scores[feature] = difference

# Find the feature with the smallest absolute difference
for feature, difference in feature_scores.items():
    if difference < smallest_difference:
        smallest_difference = difference
        least_useful_feature = feature

# Print the least useful feature and its smallest absolute difference
print("The least useful feature is:", least_useful_feature)
print("The smallest absolute difference is:", smallest_difference)

The least useful feature is: make=FIAT
The smallest absolute difference is: 0.0


In [92]:
# List of features to exclude
features_to_exclude = ['year', 'engine_hp', 'transmission_type', 'city_mpg']

smallest_difference = float('inf')
least_useful_feature = None

# Find the feature with the smallest absolute difference within features_to_exclude
for feature in features_to_exclude:
    difference = feature_scores.get(feature, float('inf'))
    if difference < smallest_difference:
        smallest_difference = difference
        least_useful_feature = feature

# Print the least useful feature and its smallest absolute difference
print("The least useful feature within features_to_exclude is:", least_useful_feature)
print("The smallest absolute difference is:", smallest_difference)

The least useful feature within features_to_exclude is: engine_hp
The smallest absolute difference is: 0.004719454640797038


## Question 6

- For this question, we'll see how to use a linear regression model from Scikit-Learn.
- We'll need to use the original column price. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
- This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
- Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

In [114]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Apply a logarithmic transformation to the 'price' column
data_with_price['price_log'] = np.log1p(data_with_price['price'])

# Split the data into training and validation sets
X = data_with_price.drop(['price', 'price_log'], axis=1)  # Exclude 'price' and 'price_log' from features
y = data_with_price['price_log']  # Use 'price_log' as the target variable

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


train_dict = X_train.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)

val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict)


# List of alpha values to try
alphas = [0, 0.01, 0.1, 1, 10]

# Initialize variables to track the best alpha and RMSE
best_alpha = None
best_rmse = float('inf')

# Fit Ridge regression models with different alpha values
for alpha in alphas:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    # Check if this model has a lower RMSE
    if rmse < best_rmse:
        best_rmse = rmse
        best_alpha = alpha

# Round the best RMSE to 3 decimal digits
best_rmse_rounded = round(best_rmse, 3)

print("The best alpha for Ridge regression is:", best_alpha)
print("The best RMSE on the validation set is:", best_rmse_rounded)

D:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The best alpha for Ridge regression is: 0
The best RMSE on the validation set is: 0.479


D:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
